In [2]:
# -*- coding: utf-8 -*-


import os
import json
import tensorflow as tf

In [3]:
def parse_fn(example):
    
    f = {
        "visit_items_index":tf.FixedLenFeature([5], tf.int64),
        "continuous_features_value":tf.FixedLenFeature([16],tf.float32),
        "next_visit_item_index":tf.FixedLenFeature([],tf.int64)
    }
    parsed = tf.parse_single_example(example,f)
    next_visit_item_index = parsed.pop("next_visit_item_index")
    return parsed, next_visit_item_index

In [4]:
def input_fn(path, parallel_num, epoch_num, batch_size):
    
    files = tf.data.Dataset.list_files(path, shuffle=True)
    dataset = files.apply(
        tf.contrib.data.parallel_interleave(
            lambda filename: tf.data.TFRecordDataset(filename),
            cycle_length=parallel_num
    ))   
    dataset = dataset.repeat(epoch_num).map(map_func=parse_fn, num_parallel_calls=parallel_num)
    dataset = dataset.prefetch(10*batch_size).batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    features , labels = iterator.get_next()
    return features, labels
    
    

In [5]:
def model_fn(features, labels, mode, params, config):
    
    visit_items_index = features['visit_items_index']  # num * 5
    continuous_features_value = features['continuous_features_value'] # num * 16
    next_visit_item_index = labels
    keep_prob = params["keep_prob"]
    embedding_size = params["embedding_size"]
    item_num = params["item_num"]
    learning_rate = params["learning_rate"]
    top_k = params["top_k"]
    deep_layers = params["deep_layers"]
    
    
    # items embedding init
    initializer = tf.initializers.random_uniform(minval=-0.5/embedding_size, maxval=0.5/embedding_size)
    partitioner = tf.fixed_size_partitioner(num_shards=embedding_size)
    item_embedding = tf.get_variable("item_embedding",
                                     [item_num, embedding_size],
                                    tf.float32,
                                    initializer=initializer,
                                    partitioner=partitioner)
    visit_items_embedding = tf.nn.embedding_lookup(item_embedding, visit_items_index) # num * 5 * embedding_size
    visit_items_average_embedding = tf.reduce_mean(visit_items_embedding, axis=1)  # num * embedding_size
    inputs = tf.concat([visit_items_average_embedding, continuous_features_value], 1)  # num * (embedding_size + 16)
    
    # dnn model
    for i in range(len(deep_layers)):
        inputs = tf.contrib.layers.fully_connected(inputs=inputs, 
                                            num_outputs=deep_layers[i],
                                            weights_initializer=tf.initializers.random_normal(mean=0.0, stddev=0.1),
                                            biases_initializer=tf.initializers.random_normal(mean=0.0, stddev=0.1))
        inputs = tf.nn.dropout(inputs, keep_prob=keep_prob, name='layer_%d' % i)
    
    # user vector 是最后一层的output
    user_vector = inputs
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        
        output_embedding = tf.nn.embedding_lookup(item_embedding, next_visit_item_index) # num * embedding_size
        logits = tf.matmul(user_vector, output_embedding, transpose_a=False, transpose_b=True)    # num * num
        yhat = tf.nn.softmax(logits)  # num * num
        cross_entropy = tf.reduce_mean(-tf.log(tf.matrix_diag_part(yhat) + 1e-16))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(cross_entropy, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=cross_entropy, train_op=train)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        
        output_embedding = tf.nn.embedding_lookup(item_embedding, next_visit_item_index)
        logits = tf.matmul(user_vector, output_embedding, transpose_a=False, transpose_b=True)
        yhat = tf.nn.softmax(logits)
        cross_entropy = tf.reduce_mean(-tf.log(tf.matrix_diag_part(yhat)+1e-16))
        return tf.estimator.EstimatorSpec(mode, loss=cross_entropy)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        
        logits_predict = tf.matmul(user_vector, item_embedding, transpose_a=False, transpose_b=True)
        yhat_predict = tf.nn.softmax(logits_predict)
        _, indices = tf.nn.top_k(yhat_predict, k=top_k, sorted=True)
        index = tf.identity(indices, name="index")
        
        predictions = {
            "user_vector":user_vector,
            "index":index,
            "item_embedding":item_embedding
        }
        export_outputs = {
            "prediction":tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions, export_outputs=export_outputs)
        


In [6]:
def build_estimator():
    params = {
        "keep_prob":0.5,
        "embedding_size":16,
        "item_num":500,
        "learning_rate":0.05,
        "top_k":2,
        "deep_layers":[64,32,16]
    }
    config = tf.estimator.RunConfig(
        model_dir="./ckpt",
        tf_random_seed=2019,
        save_checkpoints_steps=100,
        keep_checkpoint_max=5,
        log_step_count_steps=100
    )
    estimator = tf.estimator.Estimator(model_fn=model_fn, config=config, params=params)
    return estimator


In [ ]:
if __name__ == "__main__":
    
    estimator = build_estimator()
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=lambda:input_fn(
            path='./data/train.tf.records',
            parallel_num=5,
            epoch_num = 10,
            batch_size=32),
        max_steps=1600)
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn=lambda:input_fn(
            path='./data/train.tf.records',
            parallel_num=5,
            epoch_num = 1,
            batch_size=32),
        steps=15,
        start_delay_secs=1,
        throttle_secs=20
    )
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    
    features_spec = {
        "visit_items_index": tf.placeholder(tf.int64, shape=[None, 5], name="visit_items_index"),
        "continuous_features_value":tf.placeholder(tf.float32, shape=[None, 16], name="continuous_features_value")
    }
    serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(features_spec)
    estimator.export_savedmodel("./model", serving_input_receiver_fn)



W0106 20:31:31.052049 32168 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0106 20:31:31.074018 32168 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\data\util\random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0106 20:31:33.407733 32168 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/2